In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

FILEPATH = './DataSet_w_NA.xlsx'

df = pd.read_excel(FILEPATH, sheet_name="Испорченные факты")
df = df.dropna()
df = pd.pivot_table(df, values=['Продажи, руб', 'Продажи, шт', 'Повторение заказа', 'Маржинальная прибыль', 'Повторение товара'], index=["Факты.Товар ID"],
                    aggfunc={'Продажи, шт': [np.median, np.sum],
                             'Продажи, руб': np.sum,
                             'Повторение заказа': np.sum,
                             'Маржинальная прибыль': np.sum

                             })
newname = df.columns.map('_'.join)
df.columns = newname
df = df.reset_index()
total_sale = df['Продажи, руб_sum'].sum()
df['Доля'] = df['Продажи, руб_sum']/total_sale * 100
df = df.sort_values(by=('Продажи, руб_sum'), ascending=False)
df = df.assign(sum_d=df['Доля'].cumsum())

df.loc[(df['sum_d'] <= 80), 'ABC'] = 'A'
df.loc[(df['sum_d'] > 80) & (df['sum_d'] <= 95), 'ABC'] = 'B'
df.loc[(df['sum_d'] > 95), 'ABC'] = 'C'


df['Стоимость, руб'] = df['Продажи, руб_sum']/df['Продажи, шт_sum']
df['Продажи в следующем периоде, шт'] = (df['Продажи, шт_sum'] + df['Продажи, шт_median'])
df['Продажи в следующем периоде'] = (
    df['Продажи, шт_sum'] + df['Продажи, шт_median']) * df['Стоимость, руб']
total_sale_next = df['Продажи в следующем периоде'].sum()
df['Доля_будущая'] = df['Продажи в следующем периоде']/total_sale_next * 100
df = df.sort_values(by=('Продажи в следующем периоде'), ascending=False)
df = df.assign(sum_d_next=df['Доля_будущая'].cumsum())

df.loc[(df['sum_d_next'] <= 80), 'ABC_next'] = 'A'
df.loc[(df['sum_d_next'] > 80) & (df['sum_d_next'] <= 95), 'ABC_next'] = 'B'
df.loc[(df['sum_d_next'] > 95), 'ABC_next'] = 'C'

df = df.sample(frac=1)
X = df[['Стоимость, руб', 'Продажи в следующем периоде, шт']]
y = df['ABC_next']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [7]:
model = RandomForestClassifier(n_estimators=20)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

my_pred = model.predict([[30000,5]])

print(classification_report(y_test, y_pred))
print(f'Предсказанное значение ABC {my_pred}')

              precision    recall  f1-score   support

           A       0.98      0.99      0.98        81
           B       0.98      0.97      0.97        98
           C       0.99      0.99      0.99       194

    accuracy                           0.99       373
   macro avg       0.98      0.98      0.98       373
weighted avg       0.99      0.99      0.99       373

Предсказанное значение ABC ['C']


In [4]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(X_train, y_train)
y_pred_1 = svclassifier.predict(X_test)

print(classification_report(y_test, y_pred_1))

              precision    recall  f1-score   support

           A       0.90      0.94      0.92        81
           B       0.85      0.88      0.86        98
           C       0.98      0.95      0.96       194

    accuracy                           0.93       373
   macro avg       0.91      0.92      0.92       373
weighted avg       0.93      0.93      0.93       373



In [12]:
logRegression = LogisticRegression()
logRegression = logRegression.fit(X_train, y_train)
y_pred_log = logRegression.predict(X_test)

print(classification_report(y_test,y_pred_log))

              precision    recall  f1-score   support

           A       0.66      0.75      0.71        97
           B       0.53      0.42      0.47        97
           C       0.85      0.88      0.86       179

    accuracy                           0.73       373
   macro avg       0.68      0.68      0.68       373
weighted avg       0.72      0.73      0.72       373

